<a href="https://colab.research.google.com/github/Taka0007/2023-seminar-previous.term/blob/main/code/%E9%9F%B3%E5%A3%B0%E5%88%86%E9%9B%A2NMF%E5%AE%9F%E8%A3%85%E7%94%A8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from numpy import linalg as LA
from sklearn.decomposition import NMF
from scipy import signal

In [5]:
# wavファイルDL
!wget https://github.com/Taka0007/2023-seminar-previous.term/raw/main/wav-files/aiueo.wav -O aiueo.wav
file_path = '/content/aiueo.wav'

--2023-06-21 05:44:07--  https://github.com/Taka0007/2023-seminar-previous.term/raw/main/wav-files/aiueo.wav
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/Taka0007/2023-seminar-previous.term/main/wav-files/aiueo.wav [following]
--2023-06-21 05:44:08--  https://raw.githubusercontent.com/Taka0007/2023-seminar-previous.term/main/wav-files/aiueo.wav
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 32832 (32K) [audio/wav]
Saving to: ‘aiueo.wav’

aiueo.wav           100%[===================>]  32.06K  --.-KB/s    in 0s      

2023-06-21 05:44:08 (85.8 MB/s) - ‘aiueo.wav’ saved [32832/32832]



In [8]:
# 音響分離作業

#audio_data, sr = librosa.load(file_path)
file_path = '/content/Vinyl-sabi.wav'
audio_data, sr = librosa.load(file_path)
amplitude = np.abs(librosa.stft(audio_data))
matrix_fc, matrix_ct = librosa.decompose.decompose(amplitude, n_components=16, sort=True)
reconstructed_amplitude = matrix_fc.dot(matrix_ct)

print('matrix_fc')
print(matrix_fc)

print('matrix_ct')
print(matrix_ct)

print('reconstructed_amplitude')
print(reconstructed_amplitude)

<ipython-input-8-f3959dd84441>:5: UserWarning: PySoundFile failed. Trying audioread instead.
  audio_data, sr = librosa.load(file_path)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


matrix_fc
[[6.7768060e-02 3.0493718e-01 8.1774108e-02 ... 2.1419160e-03
  8.4144166e-03 2.2264682e-01]
 [7.9755709e-02 2.7046981e-01 6.0524695e-02 ... 0.0000000e+00
  3.1364083e-02 2.3789281e-01]
 [8.2662463e-02 2.4497555e-01 2.8233733e-02 ... 0.0000000e+00
  4.4371538e-02 2.5454915e-01]
 ...
 [4.6752899e-05 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.6575318e-05 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.6492616e-05 0.0000000e+00 0.0000000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]
matrix_ct
[[3.3834846e-09 3.8933200e-07 9.6388669e-07 ... 5.1396704e-01
  3.3293447e-01 2.8959706e-01]
 [1.3965027e-09 2.6958196e-09 6.8783500e-08 ... 0.0000000e+00
  0.0000000e+00 9.1483183e-03]
 [0.0000000e+00 0.0000000e+00 0.0000000e+00 ... 1.7859565e-02
  0.0000000e+00 0.0000000e+00]
 ...
 [5.0715765e-09 4.3117592e-07 1.1558599e-06 ... 0.0000000e+00
  2.9281087e-02 3.9939258e-02]
 [2.4243683e-09 1.1389510e-06 1.8264154

/usr/local/lib/python3.10/dist-packages/sklearn/decomposition/_nmf.py:1665: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [ ]:
type(fft_spec)

numpy.ndarray

In [ ]:
print(fft_spec)
print(fft_spec.shape)

[[-2.78108568e+01+0.00000000e+00j]
 [-4.17856923e+00+1.67983792e+01j]
 [ 2.63063313e+01+1.49804326e+01j]
 ...
 [ 6.29508110e-03-1.11692273e-06j]
 [ 6.29458866e-03+2.31351483e-06j]
 [ 6.29570565e-03-2.41347811e-07j]]
(54905, 1)


In [ ]:
# 複素NMFの実装
def complex_nmf(V, rank, num_iterations):
    # 行列のサイズを取得
    n, m = V.shape

    # WとHの初期化
    W = np.random.rand(n, rank)
    H = np.random.rand(rank, m)

    # 反復更新
    for i in range(num_iterations):
        # Wの更新
        numerator_W = np.abs(V) @ H.T
        denominator_W = W @ H @ H.T
        W *= numerator_W / (denominator_W + 1e-9)

        # Hの更新
        numerator_H = W.T @ np.abs(V)
        denominator_H = W.T @ W @ H
        H *= numerator_H / (denominator_H + 1e-9)

    return W, H

rank = np.linalg.matrix_rank(fft_spec)
W, H = complex_nmf(fft_spec, rank, num_iterations=100)
Ans = W @ H
# 結果の表示
print("W:")
print(W)

print("H:")
print(H)

print('fft_spec')
print(fft_spec)

print('Ans')
print(Ans)


W:
[[1.29850093e+02]
 [8.08224713e+01]
 [1.41344550e+02]
 ...
 [2.93919845e-02]
 [2.93896868e-02]
 [2.93949001e-02]]
H:
[[0.21417664]]
fft_spec
[[-2.78108568e+01+0.00000000e+00j]
 [-4.17856923e+00+1.67983792e+01j]
 [ 2.63063313e+01+1.49804326e+01j]
 ...
 [ 6.29508110e-03-1.11692273e-06j]
 [ 6.29458866e-03+2.31351483e-06j]
 [ 6.29570565e-03-2.41347811e-07j]]
Ans
[[2.78108568e+01]
 [1.73102855e+01]
 [3.02727010e+01]
 ...
 [6.29507653e-03]
 [6.29458441e-03]
 [6.29570099e-03]]


## 参考資料

[【超初心者向け】複素NMFとは？更新式を丁寧に導出。](https://tips-memo.com/complex-nmf)